In [1]:
import duckdb
import pandas as pd
from pathlib import Path



Task 1 - EDA in python

In [21]:
# duckdb database path
duckdb_path = "data/sakila.duckdb" 
Path(duckdb_path).unlink(missing_ok=True) 

# Ingest Sakila dataset into DuckDB
with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_scripts:
    conn.sql(ingest_scripts.read())

In [ ]:
description